## PostgreSQL Python: Connect To PostgreSQL Database Server

In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect("dbname=suppliers user=postgres password=ankur715")

In [3]:
conn = psycopg2.connect(host="localhost", database="ZAGDB", user="postgres", password="ankur715")

The following is the list of the connection parameters:  

- database: the name of the database that you want to connect.  
- user: the username used to authenticate.  
- password: password used to authenticate.  
- host: database server address e.g., localhost or an IP address.  
- port: the port number that defaults to 5432 if it is not provided.

The following config() function read the __database.ini__ file and returns the connection parameters. 

We put the __config()__ function in the __config.py__ file:

In [4]:
#!/usr/bin/python
from configparser import ConfigParser
 
def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

The following __connect()__ function connects to the suppliers database and prints out the PostgreSQL database version.

In [5]:
#!/usr/bin/python
import psycopg2
from config import config
 
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()
 
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
      
        # create a cursor
        cur = conn.cursor()
        
   # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
 
        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
       # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
 
 
if __name__ == '__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.2, compiled by Visual C++ build 1914, 64-bit',)
Database connection closed.


Consider building the query dynamically to ensure the number of placeholders matches your table and CSV file format. Then it's just a matter of ensuring your table and CSV file are correct, instead of checking that you typed enough ? placeholders in your code.

## Interacting with the database using psycopg2:

In [1]:
import psycopg2
conn = psycopg2.connect(host="localhost", database="ZAGDB", user="postgres", password="ankur715")
cur = conn.cursor()
cur.execute('SELECT * FROM category;')
one = cur.fetchone()
all = cur.fetchall()

In [2]:
one

('CP', 'Camping')

In [3]:
all

[('FW', 'Footwear'),
 ('CL', 'Climbing'),
 ('EL', 'Electronics'),
 ('CY', 'Cycling')]

In [38]:
tables = ["category","customer","product","region","salestransaction","soldvia","store","vendor"]
for i in range(len(tables)):
    cur.execute('SELECT * FROM {};'.format(tables[i]))
    tables[i] = cur.fetchall()

In [43]:
for i,j in enumerate(tables):
    j = pd.DataFrame(j)
    print(j.head())

    0            1
0  CP      Camping
1  FW     Footwear
2  CL     Climbing
3  EL  Electronics
4  CY      Cycling
         0     1      2
0  1-2-333  Tina  60137
1  2-3-444  Tony  60611
2  3-4-555   Pam  35401
3  4-5-666  Elly  47374
4  5-6-777  Nora  60640
     0          1       2   3   4
0  1X1    Zzz Bag  100.00  PG  CP
1  2X2  Easy Boot   70.00  MK  FW
2  3X3  Cosy Sock   15.00  MK  FW
3  4X4  Dura Boot   90.00  PG  FW
4  5X5  Tiny Tent  150.00  MK  CP
   0            1
0  C  Chicagoland
1  T     Tristate
2  I      Indiana
3  N        North
      0        1   2           3
0  T111  1-2-333  S1  2013-01-01
1  T222  2-3-444  S2  2013-01-01
2  T333  1-2-333  S3  2013-01-02
3  T444  3-4-555  S3  2013-01-02
4  T555  2-3-444  S3  2013-01-02
     0     1  2
0  1X1  T111  1
1  2X2  T222  1
2  3X3  T333  5
3  1X1  T333  1
4  4X4  T444  1
    0      1  2
0  S1  60600  C
1  S2  60605  C
2  S3  35400  T
3  S4  60640  C
4  S5  46307  T
    0                   1
0  PG       Pacifica Gear
1  MK 

## Creating a table:

Similar to running a SELECT query, we will write the command as a string and pass it to the execute() method.

In [7]:
cur.execute("""
CREATE TABLE user_accounts(
name text PRIMARY KEY,
department text,
title text,
telephonenumber text,
samaccountname text)
""")

Whenever we open a Connection in psycopg2, a new transaction will automatically be created. All queries run up until the commit method is called will be placed into the same transaction block. When commit is called, the PostgreSQL engine will run all the queries at once.

To commit our changes and create the users table from before, all we need to do is to run the `commit()` method at the end of our transaction.

In [8]:
conn.commit()

## Inserting the data:

In [9]:
insert_query = "INSERT INTO user_accounts VALUES {}".format("('bob', 'IT', 'engineer', ' +44', 'bobit24')")
cur.execute(insert_query)
conn.commit()

In [27]:
cur.execute("INSERT INTO user_accounts VALUES (%s, %s, %s, %s, %s)", ('abob', 'aIT', 'aengineer', '+44', 'abobit24'))
conn.commit()

In [ ]:
import csv
cur = conn.cursor()
with open('user_accounts.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # Skip the header row.
    def convert(list):
        return tuple(list)
    
    for row in reader:
        row = convert(row)
        cur.execute(
        "INSERT INTO users VALUES {}".format(row)
        )
    conn.commit()

## Copying the data:

In [ ]:
with open('user_accounts.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    next(f) # Skip the header row.
    cur.copy_from(f, 'user_accounts', sep=',')
    conn.commit()